%%<br>
mports

In [2]:
import os, sys, json, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import glob
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import lxml
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats

import requests

from bs4 import BeautifulSoup,SoupStrainer
from io import StringIO

In [3]:
soup_url = r'https://ballotpedia.org/State_Legislative_Districts'
# %%

In [4]:
all_districts = []
response = requests.get(soup_url, verify = False)
soup = BeautifulSoup(response.content, 'html.parser')
state_districts = soup.find_all("a", href = True)

In [5]:
h_refs = []
for url in state_districts:
    if 'state legislative districts' in str(url):
        # print(url)
        base = "https://ballotpedia.org/"
        full_url = base + str(url.text).replace(' ',"_")
        h_refs.append(full_url)

print(*h_refs, sep="\n")

In [6]:
# Fetches all of the districts (commented out until needed to repull)
# for ref in h_refs:
#     url = ref
    
#     page = requests.get(url)
    
#     os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\txt files for troubleshooting')

#     # Write the page's text content to a file
#     # with open('output_soup_strainer.txt', "w", encoding="utf-8") as f:
#     #     f.write(page.text)
#     # print(page.content)
#     table_strainer = SoupStrainer('table', id='officeholder-table')
#     page_soup = BeautifulSoup(page.content, 'html.parser', parse_only=table_strainer)

#     # print(page_soup.content)
#     # print(type(page_soup))
#     districts = page_soup.find_all("a")
#     total_districts = []
#     # print(page_soup.prettify())
#     for d in districts:
#         total_districts.append(d.text)
#         # print(d.text)
#     all_districts.extend(total_districts)


In [7]:


# # Save data to JSON file
# os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\json save data')
# with open(f"all_districts_{str(date.today()).replace('-', '_')}.json", "w") as f:
#     json.dump(all_districts, f)
#     save_file_name = f.name
#     print(save_file_name)

In [8]:
#loading districts webscraping data
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\json save data')
json_files = glob.glob('*.json')

max_mtime = 0
for dirname,subdirs,files in os.walk("."):
    for fname in files:
        full_path = os.path.join(dirname, fname)
        mtime = os.stat(full_path).st_mtime
        if mtime > max_mtime:
            max_mtime = mtime
            max_dir = dirname
            max_file = fname
save_file_name = max_file


In [9]:



with open(f'{save_file_name}', "r") as f:
    all_districts = json.load(f)


In [10]:
state_intitals = []
for state in all_districts:
    state_match = re.findall(state_pat, state)[0]
    ref_lookup = state_ref.get(str(state_match))
    state_intitals.append(ref_lookup)

In [11]:
def filter_row(row, string_column):
    # Check conditions using an if-else statement
    if re.search(r'[Hh]ouse|[Rr]epresentative', str(row[string_column])):
        return "House"
    elif re.search(r'[Ss]enate', str(row[string_column])):
        return "Senate"
    else:
        return 'Unknown'


In [12]:
districts_w_intials = pd.DataFrame({'state_abbreviation': state_intitals,'district_string': all_districts})
districts_w_intials = districts_w_intials[~districts_w_intials['district_string'].str.contains(r'[Hh]istorical|9[AB]{1}', regex=True)]
thi_state_districts = districts_w_intials[districts_w_intials['state_abbreviation'].isin(thi_states)]

thi_state_districts.reset_index(inplace=True, drop=True)


thi_state_districts['chamber'] = thi_state_districts.apply(
    filter_row, args=('district_string',), axis=1
)

thi_state_districts["district"] = thi_state_districts["district_string"].str.extractall(r"(\d+)").unstack().fillna('').apply(' '.join, 1)


# Apply the filtering function row-wise
# filtered_df = thi_state_districts[thi_state_districts.apply(filter_row, args=('district_string',), axis=1)]
# thi_state_districts['chamber'] = ""
# thi_state_districts['chamber'] = ""
thi_state_districts

,state_abbreviation,district_string,chamber,district
0,AL,Alabama State Senate District 1,Senate,1
1,AL,Alabama State Senate District 2,Senate,2
2,AL,Alabama State Senate District 3,Senate,3
3,AL,Alabama State Senate District 4,Senate,4
4,AL,Alabama State Senate District 5,Senate,5
...,...,...,...,...
1989,WV,West Virginia House of Delegates District 96,House,96
1990,WV,West Virginia House of Delegates District 97,House,97
1991,WV,West Virginia House of Delegates District 98,House,98
1992,WV,West Virginia House of Delegates District 99,House,99


In [ ]:
#create pk for leg seats from ballot pedia

leg_keys, leg_keys_dupes = create_pk(thi_state_districts, 'district', 'chamber')

# leg_lookup = pd.concat([leg_keys_wseats,leg_keys_dupes_wseats])
leg_lookup = pd.concat([leg_keys,leg_keys_dupes])


#uncomment for help troubleshooting
#  os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data')
# leg_lookup.to_csv('leg_lookup.csv')




In [46]:

#getting and creating key for all leg files

all_leg_data = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\all_legs_files_2024_11_22.csv" 

all_leg_df = pd.read_csv(all_leg_data)


all_leg_wkey, all_leg_dupes_wkey = create_pk(all_leg_df, 'district', 'Chamber')
# all_leg_wkey, all_leg_dupes_wkey = add_seats(all_leg_wkey, all_leg_dupes_wkey, 'First Name', 'Last Name', keep_names = True)
all_leg_lookup = pd.concat([all_leg_wkey, all_leg_dupes_wkey])


#uncomment for help troubleshooting
# os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data')
# all_leg_lookup.to_csv('all_leg_lookup.csv')

all_leg_lookup_for_merge = all_leg_lookup.loc[:,['primary_key', 'First Name', 'Last Name']]
leg_lookup_for_merge = leg_lookup.loc[:,['primary_key','state_abbreviation']] 


merge_1 = pd.merge(leg_lookup_for_merge, all_leg_lookup_for_merge, how='left', left_on='primary_key', right_on='primary_key')

#gets multiseat districts
merge_1_dupes = merge_1[merge_1.duplicated(subset='primary_key',keep=False)]
#gets only non multiseat districts
merge_1_nodupes = merge_1[~merge_1.duplicated(subset='primary_key',keep=False)]


In [54]:
# these are fine
merge_wseats = add_seats(df = merge_1_nodupes, keep_names = True)

merge_1_dupes = merge_1_dupes.drop_duplicates()



full_pk,primary_key,state_abbreviation,First Name,Last Name,seat_num


In [61]:



#truncated for lookup ease and cleaner look
leg_keys_dupes_for_merge = leg_keys_dupes.loc[:,['primary_key','state_abbreviation']] 

#full seats list for ref to merge left
# seats_ref = add_seats(df_duplicates = leg_keys_dupes_for_merge)

# merge_dupes_wseats = add_seats(df_duplicates=merge_1_dupes, keep_names = True)
#merges the absolute positions with the values found from legislator files
merge_2 = pd.merge(leg_keys_dupes_for_merge, merge_1_dupes, how='left', left_on='primary_key', right_on='primary_key')
merge_2 = merge_2.drop_duplicates()

merge_2 = add_seats(df_duplicates = merge_2, keep_names = True)



to_combine = [merge_wseats, merge_2]
full_ref = pd.concat(to_combine)
full_ref.reset_index(inplace = True, drop = True)

leg_keys_wseats, leg_keys_dupes_wseats = add_seats(df = leg_keys, df_duplicates = leg_keys_dupes, keep_names = True)
ref_back = pd.concat([leg_keys_wseats, leg_keys_dupes_wseats])
ref_back.reset_index(inplace = True, drop = True)

print(f'full ref columns: {full_ref.columns}')
print(f'ref back columns: {ref_back.columns}')




#reference for checking vacancies

#from final merge also should be the final lookup##
leg_lookup_df = full_ref.loc[:,['full_pk', 'primary_key','First Name', 'Last Name']]

#from intial data
ref_back = ref_back.loc[:,['full_pk', 'state_abbreviation']]
# full_ref = full_ref.drop(['state_abbreviation_x', 'state_abbreviation_y'], axis = 1)


# test = pd.merge(ref_back, full_ref, how='left', left_on='full_pk', right_on='full_pk')

leg_lookup_df.to_csv(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors\leg_lookup_df.csv', index = False)


full_pk,primary_key,state_abbreviation_x,state_abbreviation_y,First Name,Last Name,seat_num
full_pk,primary_key,state_abbreviation,district_string,chamber,district,state_code,chamber_code,district_code,seat_num
full_pk,primary_key,state_abbreviation,district_string,chamber,district,state_code,chamber_code,district_code,seat_num
full ref columns: Index(['full_pk', 'primary_key', 'state_abbreviation', 'First Name',
       'Last Name', 'seat_num', 'state_abbreviation_x',
       'state_abbreviation_y'],
      dtype='object')
ref back columns: Index(['full_pk', 'primary_key', 'state_abbreviation', 'district_string',
       'chamber', 'district', 'state_code', 'chamber_code', 'district_code',
       'seat_num'],
      dtype='object')


%%splitting up district numbers from rest of string

In [ ]:

def create_pk(df,column):
    lengths = []
    df.loc[:,'state_code'] = np.nan
    df.loc[:,'chamber_code'] = np.nan
    df.loc[:,'district'] = np.nan
    df.loc[:,'primary_key'] = np.nan
    for i,j in enumerate(df[f'{column}']):
        # print(str(j))
        # print(row)
        district_raw = re.split(r'\s(?=District)', str(j))
        match = re.findall(r'\s\d+', str(district_raw))[0]
        match = match.strip()
        if len(match) == 2:
            district_code = '0' + str(match)
        elif len(match) == 1:
            district_code = '00'+str(match)
        else:
            district_code = str(match)
        district_len = len(match)
        lengths.append(district_len)
        ext_state = df.loc[i,'state_abbreviation']
        state_code = state_coding.get(ext_state)
        if 'house' in str(j).lower():
            chamber_code = '0'
        elif 'senate' in str(j).lower():
            chamber_code = '1'
        else:
            print(f'unknown chamber: {str(j)}')
            break
        
        # display_markdown(f'#### {ext_state} - {chamber_code} - {district_raw}', raw=True)
        key_code = f'{state_code}{chamber_code}{district_code}'
        
        
        df.loc[i,'state_code'] = state_code
        df.loc[i,'chamber_code'] = chamber_code
        df.loc[i,'district'] = match
        df.loc[i,'primary_key'] = key_code
    return df

In [ ]:
#dont touch original

lengths = []
thi_state_districts['state_code'] = np.nan
thi_state_districts['chamber_code'] = np.nan
thi_state_districts['district'] = np.nan
thi_state_districts['primary_key'] = np.nan
for i,j in enumerate(thi_state_districts['district_string']):
    # print(str(j))
    # print(row)
    district_raw = re.split(r'\s(?=District)', str(j))
    match = re.findall(r'\s\d+', str(district_raw))[0]
    match = match.strip()
    if len(match) == 2:
        district_code = '0' + str(match)
    elif len(match) == 1:
        district_code = '00'+str(match)
    else:
        district_code = str(match)
    district_len = len(match)
    lengths.append(district_len)
    ext_state = thi_state_districts.loc[i,'state_abbreviation']
    state_code = state_coding.get(ext_state)
    if 'house' in str(j).lower():
        chamber_code = '0'
    elif 'senate' in str(j).lower():
        chamber_code = '1'
    else:
        print(f'unknown chamber: {str(j)}')
        break
    
    # display_markdown(f'#### {ext_state} - {chamber_code} - {district_raw}', raw=True)
    key_code = f'{state_code}{chamber_code}{district_code}'
    
    
    thi_state_districts.loc[i,'state_code'] = state_code
    thi_state_districts.loc[i,'chamber_code'] = chamber_code
    thi_state_districts.loc[i,'district'] = match
    thi_state_districts.loc[i,'primary_key'] = key_code

In [ ]:
    
    
# %% takes duplicates of primary keys and assigns a seat num

eat num of 00 means there is only one seat (no multi-member districts)

In [ ]:
thi_state_districts['count'] = thi_state_districts['primary_key'].map(thi_state_districts['primary_key'].value_counts())
thi_state_districts['seat_num'] = thi_state_districts.groupby('primary_key').cumcount() + 1
thi_state_districts.loc[thi_state_districts['count'] == 1, 'seat_num'] = 0
thi_state_districts = thi_state_districts.drop(columns='count')
thi_state_districts['seat_num'] = thi_state_districts['seat_num'].apply(lambda x: f'{x:02d}')
thi_state_districts['final_primary_key'] = thi_state_districts['primary_key'] + thi_state_districts['seat_num']
# %%

In [ ]:
# %%

In [ ]:
duplicate = thi_state_districts[thi_state_districts.duplicated('primary_key')]

    # print(str(j))
    # print(district_len)
    

print(max(lengths))

In [ ]:
    # print(line.split('District')[-1])

%%

In [ ]:
print(*all_districts, sep='\n')

In [ ]:
[print(x) for x in thi_state_districts]

    # states.append(str(state_match))
# %%

In [ ]:
states = sorted(list(set(states)))
print(len(states))
print(*states, sep="\n")

In [ ]:
for i,j in enumerate(states):
    code = str(i+1)
    if len(code) == 1:
        code = "0"+code
    df = pd.DataFrame({'state': [j], 'code':[code]})
    print(df.to_string())

%%

In [ ]:
    
    break
    print(page.status_code)
    print(page.content[:500])  # Preview the content
    
    # df_list = pd.read_html(page_soup.prettify())
    # print(len(df_list))
    # print(type(df_list))

In [ ]:
    # print(df)

    # dis_soup = BeautifulSoup(response.content, 'html.parser')
    # ditricts_tags = dis_soup.find_all("td")
    # for d in ditricts_tags:
    #     print(d)
    
	

%%